# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,283.279192,-0.636653,1234,282.642539,283.915845,565.285079,567.831690
6,937.720573,0.253327,1234,937.973900,937.467246,1875.947801,1874.934493
7,223.249371,0.920734,1234,224.170105,222.328637,448.340210,444.657273
8,180.630604,0.480585,1234,181.111189,180.150019,362.222378,360.300038
9,606.419277,0.565350,1234,606.984628,605.853927,1213.969255,1211.707853
10,361.065039,0.385899,1234,361.450937,360.679140,722.901874,721.358280
11,284.070307,-0.870115,1234,283.200192,284.940421,566.400384,569.880842
12,347.889347,0.177798,1234,348.067145,347.711549,696.134289,695.423097
13,460.571852,0.757024,1234,461.328876,459.814828,922.657752,919.629656
14,88.078795,0.571553,1234,88.650348,87.507242,177.300696,175.014483


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-282.642539,283.915845
6,-937.973900,937.467246
7,-224.170105,222.328637
8,-181.111189,180.150019
9,-606.984628,605.853927
10,-361.450937,360.679140
11,-283.200192,284.940421
12,-348.067145,347.711549
13,-461.328876,459.814828
14,-88.650348,87.507242


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-282.642539,283.915845
6,-937.973900,937.467246
7,-224.170105,222.328637
8,-181.111189,180.150019
9,-606.984628,605.853927
10,-361.450937,360.679140
11,-283.200192,284.940421
12,-348.067145,347.711549
13,-461.328876,459.814828
14,-88.650348,87.507242


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-282.64253947,  283.9158451 ],
       [-937.97390036,  937.46724628],
       [-224.1701051 ,  222.32863672],
       [-181.11118877,  180.15001889],
       [-606.98462771,  605.85392673],
       [-361.45093717,  360.67913991],
       [-283.20019208,  284.94042119],
       [-348.06714474,  347.71154864],
       [-461.32887609,  459.81482787],
       [ -88.65034799,   87.50724151]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,563.285079,-567.831690
6,1873.947801,-1874.934493
7,446.340210,-444.657273
8,360.222378,-360.300038
9,1211.969255,-1211.707853
10,720.901874,-721.358280
11,564.400384,-569.880842
12,694.134289,-695.423097
13,920.657752,-919.629656
14,175.300696,-175.014483


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.283279,0.181674
6,0.937721,0.626664
7,0.223249,0.960367
8,0.180631,0.740292
9,0.606419,0.782675
10,0.361065,0.692949
11,0.284070,0.064943
12,0.347889,0.588899
13,0.460572,0.878512
14,0.088079,0.785777


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,283.279192,-0.636653
6,937.720573,0.253327
7,223.249371,0.920734
8,180.630604,0.480585
9,606.419277,0.565350
10,361.065039,0.385899
11,284.070307,-0.870115
12,347.889347,0.177798
13,460.571852,0.757024
14,88.078795,0.571553


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
